In [ ]:
# install pyldavis
!pip install pyldavis
# imports
!pip install gensim pyLDAvis
! python3 -m spacy download en_core_web_sm

In [1]:
import pandas as pd
import numpy as np 

In [2]:
import string
!pip install spacy
import spacy
import nltk 

  Using cached spacy-3.7.4-cp39-cp39-win_amd64.whl (12.2 MB)
  Using cached srsly-2.4.8-cp39-cp39-win_amd64.whl (483 kB)
  Using cached catalogue-2.0.10-py3-none-any.whl (17 kB)
  Using cached wasabi-1.1.2-py3-none-any.whl (27 kB)
  Using cached spacy_loggers-1.0.5-py3-none-any.whl (22 kB)
  Using cached typer-0.9.4-py3-none-any.whl (45 kB)
  Using cached thinc-8.2.3-cp39-cp39-win_amd64.whl (1.5 MB)
  Using cached preshed-3.0.9-cp39-cp39-win_amd64.whl (122 kB)
  Using cached weasel-0.3.4-py3-none-any.whl (50 kB)
  Using cached langcodes-3.3.0-py3-none-any.whl (181 kB)
  Using cached murmurhash-1.0.10-cp39-cp39-win_amd64.whl (25 kB)
  Using cached spacy_legacy-3.0.12-py2.py3-none-any.whl (29 kB)
  Using cached pydantic-2.6.4-py3-none-any.whl (394 kB)
  Using cached annotated_types-0.6.0-py3-none-any.whl (12 kB)
  Using cached typing_extensions-4.11.0-py3-none-any.whl (34 kB)
  Using cached pydantic_core-2.16.3-cp39-none-win_amd64.whl (1.9 MB)
  Using cached confection-0.1.4-py3-none-any

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
anaconda-project 0.11.1 requires ruamel-yaml, which is not installed.


In [3]:
import gensim
from gensim import corpora 

In [4]:
import matplotlib.pyplot as plt

In [8]:
import pyldavis
import pyldavis.gensim_models

ModuleNotFoundError: No module named 'pyldavis'

In [9]:
nltk.download('wordnet')
from nltk.corpus import wordnet as wn
nltk.download('stopwords')
from nltk.corpus import stopwords
import spacy.cli
spacy.cli.download("en_core_web_md")
import en_core_web_md
# fetch yelp review dataset and clean it
yelp_review = pd.read_csv('/content/yelp.csv')
yelp_review.head()
# print number of document and topics
print(len(yelp_review))
print("Unique Business")
print(len(yelp_review.groupby('business_id')))
print("Unique User")
print(len(yelp_review.groupby('user_id')))

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\preet\AppData\Roaming\nltk_data...
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\preet\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_md')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


FileNotFoundError: [Errno 2] No such file or directory: '/content/yelp.csv'

In [ ]:
# clean the document and remove punctuation
def clean_text(text):
    delete_dict = {sp_char: '' for sp_char in string.punctuation}
    delete_dict[' '] =' '
    table = str.maketrans(delete_dict)
    text1 = text.translate(table)
    textArr= text1.split()
    text2 = ' '.join([w for w in textArr if ( not w.isdigit() and ( not w.isdigit() and len(w)>3))])
return text2.lower()

In [ ]:
yelp_review['text'] = yelp_review['text'].apply(clean_text)
yelp_review['Num_words_text'] = yelp_review['text'].apply(lambda x:len(str(x).split())) 

In [ ]:
print('-------Reviews By Stars --------')
print(yelp_review['stars'].value_counts())
print(len(yelp_review))
print('-------------------------')
max_review_data_sentence_length = yelp_review['Num_words_text'].max()


In [ ]:
# print short review (
mask = (yelp_review['Num_words_text'] < 100) & (yelp_review['Num_words_text'] >=20)
df_short_reviews = yelp_review[mask]
df_sampled = df_short_reviews.groupby('stars')
    .apply(lambda x: x.sample(n=100)).reset_index(drop = True)


In [ ]:
print('No of Short reviews')
print(len(df_short_reviews))


In [ ]:
# function to remove stopwords
def remove_stopwords(text):
    textArr = text.split(' ')
    rem_text = " ".join([i for i in textArr if i not in stop_words])
    return rem_text

In [ ]:
# remove stopwords from the text
stop_words = stopwords.words('english')
df_sampled['text']=df_sampled['text'].apply(remove_stopwords)


In [ ]:
# perform Lemmatization
lp = en_core_web_md.load(disable=['parser', 'ner'])
def lemmatization(texts,allowed_postags=['NOUN', 'ADJ']):
    output = []
    for sent in texts:
            doc = nlp(sent)
            output.append([token.lemma_
                            for token in doc if token.pos_ in allowed_postags ])
    return output
text_list=df_sampled['text'].tolist()
print(text_list[2])
tokenized_reviews = lemmatization(text_list)
print(tokenized_reviews[2])


In [ ]:
# convert to document term frequency:
dictionary = corpora.Dictionary(tokenized_reviews)
doc_term_matrix = [dictionary.doc2bow(rev) for rev in tokenized_reviews]

In [ ]:
# Creating the object for LDA model using gensim library
LDA = gensim.models.ldamodel.LdaModel


In [ ]:
# Build LDA model
lda_model = LDA(corpus=doc_term_matrix, id2word=dictionary,
                num_topics=10, random_state=100,
                chunksize=1000, passes=50,iterations=100)
# print lda topics with respect to each word of document
lda_model.print_topics()

In [ ]:
# calculate perplexity and coherence
print('\Perplexity: ', lda_model.log_perplexity(doc_term_matrix, total_docs=10000))

In [ ]:
# calculate coherence
coherence_model_lda = CoherenceModel(model=lda_model,
                                    texts=tokenized_reviews, dictionary=dictionary ,
                                    coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('Coherence: ', coherence_lda) 

In [ ]:
# Now, we use pyLDA vis to visualize it
pyLDAvis.sklearn.prepare(lda_tf, dtm_tf, tf_vectorizer)